In [1]:
%matplotlib inline

In [2]:
# load packages
import matplotlib.pylab as plt
import numpy as np
import time
def myfloat(b):
    try: 
        b = float(b)
    except ValueError:
        b = [b for b in b.split('-').split('+') if b]
        b = float(b[0])*10**(-float(b[1]))
    return b


In [5]:
ncol = 20
nrow = 100
tmax = 100.
dt = .05
nprt =  int(tmax/100/dt)
slope = 30.
epsh = 0.005
dx = .1
hinit = 0
prate  = 0.07/3600.
xni = 0.03  # interspace roughness
ksati =  0.0 #0.019/3600.  # interspace ksat
Soi = .0   # interspace sorptivity (m/s)

In [6]:
def write_coords(fname = 'coords', ncol = ncol, nrow = nrow, dx = dx, 
                 slope = slope, xni = xni, ksati = ksati, Soi = Soi):        
    npt = (ncol+1)*(nrow+1)  # number of points
    ne = nrow*ncol  # number of edges
    nbcell = 2*ncol + 2*nrow - 4  # number of boundary cells

    xdum = np.arange(1, (ncol+1)*dx+1 - 1e-10, dx )
    ydum = np.arange(1, (nrow+1)*dx+1 - 1e-10, dx )
    ydum, xdum = np.meshgrid(ydum, xdum)

    zmax = slope/100.*(np.max(ydum) - np.min(ydum))
    zdum = np.linspace(0, zmax, nrow+1)
    zdum = np.tile(zdum, [ncol+1]).reshape([ncol+1, nrow+1])
    
    xndum = np.ones_like(zdum)*xni    
    # xndum[:, :40]  = 0.1
    
    ksat_dum = np.ones_like(zdum)*ksati    
    Sodum = np.ones_like(zdum)*Soi    
    
    x = np.zeros(npt + 1)
    y = np.zeros(npt + 1)
    z = np.zeros(npt + 1)
    xn = np.zeros(npt + 1)
    satk = np.zeros(npt + 1)
    So = np.zeros(npt + 1)
    x[1:] = xdum.ravel()
    y[1:] = ydum.ravel()
    z[1:] = zdum.ravel()
    xn[1:] = xndum.ravel()
    satk[1:] = ksat_dum.ravel()
    So[1:] = Sodum.ravel()

    # print (np.max(zdum) - np.min(zdum))/(np.max(ydum) - np.min(ydum))*100
    # (ncol+1) by (nrow+1)  -  node numbers
    nodes = np.arange(1, npt+1, dtype = int).reshape([ncol+1, nrow+1])

    nop = np.zeros([ncol+1, nrow+1, 4], dtype = int)
    for j in range(ncol):
        for k in range(nrow):
            nop[j+1, k+1] =  nodes[j,k], nodes[j+1, k], nodes[j+1,k+1], nodes[j,k+1]
            
    f = open(fname, 'w')
    f.write('{0:<13}   {1:<13}\n'.format(npt, ne))

    # write x, y, z
    for n in range(1, npt+1):
        f.write('{0:<13.6f} {1:<13.6f} {2:<13.6f} {3:<13.6e} {4:<13.6e} {5:<13.6e} \n'.format(
                    x[n],y[n],z[n],xn[n],satk[n],So[n])) 

    # write node numbers  
    for j in range(1, ncol+1):
        for k in range(1, nrow+1):
            n1 = nop[j, k, 0] 
            n2 = nop[j, k, 1]       
            n3 = nop[j, k, 2]        
            n4 = nop[j, k, 3] 
            f.write('{0:<10} {1:<10}  {2:<10} {3:<10}\n'.format(n1, n2, n3, n4)) 
    f.close()  

    # get cell center values:
    xcc  = np.zeros([ncol+2, nrow+2])    
    ycc  = np.zeros([ncol+2, nrow+2])
    zcc  = np.zeros([ncol+2, nrow+2])
    xncc  = np.zeros([ncol+2, nrow+2])
    satkc = np.zeros([ncol+2, nrow+2])
    Soc = np.zeros([ncol+2, nrow+2])    
    for j in range(1, ncol+1):
        for k in range(1, nrow+1):
            n1 = nop[j, k, 0] 
            n2 = nop[j, k, 1]       
            n3 = nop[j, k, 2]        
            n4 = nop[j, k, 3]  
            xcc[j,k] = 0.25*(x[n1] + x[n2] + x[n3] + x[n4])  
            ycc[j,k] = 0.25*(y[n1] + y[n2] + y[n3] + y[n4])
            zcc[j,k] = 0.25*(z[n1] + z[n2] + z[n3] + z[n4])   
            xncc[j,k] = 0.25*(xn[n1] + xn[n2] + xn[n3] + xn[n4])   
            satkc[j,k] = 0.25*(satk[n1] + satk[n2] + satk[n3] + satk[n4])   
            Soc[j,k] = 0.25*(So[n1] + So[n2] + So[n3] + So[n4])   
    return  nop, x, y, z, xcc, ycc, zcc, xncc, satk, Soc


In [7]:
def write_dryin(fname = 'dryin.dat', ncol = ncol, nrow = nrow, dt = dt, 
                 tmax = tmax, prate = prate, nprt = nprt, epsh = epsh,
                 hinit = hinit):
    inum = np.zeros([ncol+1, nrow+1], dtype = int)
    inum[1:, 1] = 1
    inum[1:, -1]= 1
    inum[1, 1:] = 1
    inum[-1, 1:] = 1
    inum[1, 1] = 2
    inum[1, -1] = 2
    inum[-1, -1] = 2
    inum[-1, 1] = 2
    
    ipos = np.zeros( [ncol+1, nrow+1, 2], dtype = int)
    # bottom boundary
    ipos[2:-1, 1,0] = 1
    ipos[1, 1,1] = 1
    ipos[-1, 1,1] = 1

    # right boundary
    ipos[-1, 1:-1, 0] = 2
    ipos[-1, -1,1] = 2

    # left boundary
    ipos[1, 1:, 0] = 4

    # top boundary
    ipos[2:, -1,0] = 3
    ipos[1, -1,1] = 3
    
    itype = np.zeros([ncol+1, nrow+1, 2], dtype = int)
    # bottom boundary
    itype[2:-1, 1,0] = 0
    itype[1, 1,1] = 0
    itype[-1, 1,1] = 0

    # right boundary
    itype[-1, 1:-1, 0] = 1
    itype[-1, -1,1] = 1

    # left boundary
    itype[1, 1:,0] = 1

    # top boundary
    itype[2:, -1,0] = 1
    itype[1, -1,1] = 1
    
    npt = (ncol+1)*(nrow+1)  # number of points
    ne = nrow*ncol  # number of edges
    nbcell = 2*ncol + 2*nrow - 4  # number of boundary cells

    f = open(fname, 'w')
    f.write('gravity     dt        tmax      xsplit   \n')
    f.write('9.806d0     {0}       {1}     100.d0   \n'.format(dt, tmax))
    f.write(' epsh        beta       prate \n')  
    f.write('{0}d0     2.d0       {1:6e}  \n'.format(epsh, prate))
    #     f.write('0.0025d0    2.d0   {0}d0 \n'.format(prate))
    f.write('xk          ainflt       binflt           tc           cappa \n')
    # xk = 3.9217d-4
    f.write('3.9217d-4   0.5d0       2.65d-7          0.d0        0.99d0 \n')
    f.write('istart     imass      ifront         print interval  \n')
    f.write(' 0         1          1               {0} \n'.format(nprt))
    f.write('number of boundary cell \n') 
    f.write('  {0} \n'.format(nbcell))
    f.write(' j    k          inum    itype             ipos \n')
    # f.write(' j \t k \tinum    itype \t\t ipos')
    j = 1
    for k in range(1, nrow+1):
        if inum[j, k] == 2:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<8} {4:<9} {5:<8} {6:<6} \n'.format(
                        j, k, inum[j, k], itype[j, k, 0], itype[j, k, 1], 
                         ipos[j, k, 0], ipos[j, k, 1]))
        else:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<18} {4:<10}   \n'.format(
                         j, k, inum[j, k],  itype[j, k, 0],  ipos[j, k, 0], ))

    for j in range(2, ncol+1):
        if inum[j, k] == 2:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<8} {4:<9} {5:<8} {6:<6} \n'.format(
                        j, k, inum[j, k], itype[j, k, 0], itype[j, k, 1], 
                         ipos[j, k, 0], ipos[j, k, 1]))
        else:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<18} {4:<10}   \n'.format(
                         j, k, inum[j, k],  itype[j, k, 0],  ipos[j, k, 0], ))

    for k in range(nrow-1,0,-1):
        if inum[j, k] == 2:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<8} {4:<9} {5:<8} {6:<6} \n'.format(
                        j, k, inum[j, k], itype[j, k, 0], itype[j, k, 1], 
                         ipos[j, k, 0], ipos[j, k, 1]))
        else:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<18} {4:<10}   \n'.format(
                         j, k, inum[j, k],  itype[j, k, 0],  ipos[j, k, 0], ))
            
    for j in range(ncol-1,1,-1):
        if inum[j, k] == 2:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<8} {4:<9} {5:<8} {6:<6} \n'.format(
                        j, k, inum[j, k], itype[j, k, 0], itype[j, k, 1], 
                         ipos[j, k, 0], ipos[j, k, 1]))
        else:
            f.write( '{0:<5} {1:<13} {2:<7} {3:<18} {4:<10}   \n'.format(
                         j, k, inum[j, k],  itype[j, k, 0],  ipos[j, k, 0], ))

    kbeg = np.ones(ncol+1, dtype = int)
    kend = np.ones(ncol+1, dtype = int)*nrow
   
    f.write('ncol\n')
    f.write("{0}\n".format(ncol))
    f.write('j     kbeg          kend \n')
    for j in range(1, ncol+1):
        f.write( '{0:>5}  {1:>5} {2:>13}   \n'.format(
                    j, kbeg[j],kend[k] ))

    f.write('h0l      u0l    v0l   \n ')
    f.write('{0}d0     0.0    0.0  \n '.format(hinit))
    f.write('h0r      u0r    v0r  \n ')
    f.write('{0}d0     0.0    0.0  \n '.format(hinit))
    f.close()
    return inum, ipos, itype

In [8]:
from commands import getoutput as cmd
cmd("gfortran dryt.for")

''

In [18]:
tmax = 100.

prates  = [0.01/3600, 0.03/3600, 0.07/3600, 0.10/3600]

tmaxes = []
cfls = []
runtimes = []

for p, prate in enumerate(prates):
    start_time = time.time()
    inum, ipos, itype = write_dryin(fname = 'dryin.dat', 
                                    epsh = epsh, hinit = .001)
    nop, x, y, z, xc, yc, zc, xnc, satkc, Soc = write_coords(
        fname='coords', slope = slope)
    a = cmd("./a.out")
    t = []
    f =  open("output/time.out", 'r')
    f.next()
    for line in f:
        a = (line.strip().split(" "))
        a = [b for b in a if b]
        t.append(float(a[0]))

    tmaxes.append(t[-1])
    
    f =  open("output/cfl.out", 'r')
    cfl = []
    ta = []
    for line in f:
        a = line.strip().split(" ")
        a = [np.float(b) for b in a if b]
        ta.append(a[0])
        cfl.append(a[1])

    cfls.append(np.max(np.array(cfl)))
    runtimes.append( (time.time() - start_time))
    print t[-1], 

100.05 100.05 100.05 100.05


In [19]:
import pandas as pd
cols = pd.Series(['{0:.2f} '.format(dum*3600) for dum in prates], name = 'prate')

df = pd.DataFrame({'tmax': ['{0:.1f}'.format(dum) for dum in tmaxes],  
              'runtime': ['{0:.2f}'.format(dum) for dum in runtimes],
              'CFL': ['{0:.2f}'.format(dum) for dum in cfls]},
               index = cols).T

In [21]:
fldrstr = "slope={1:.2f} prate={2:.1e} dx={3:.1} dt={0:.2f}".format( dt, slope,prate*100, dx)
fmtstr = "xn={1}".format( epsh, xni)
print  fldrstr, fmtstr
df

slope=30.00 prate=2.8e-03 dx=0.1 dt=0.05 xn=0.03


prate,0.01,0.03,0.07,0.10
CFL,0.08,0.08,0.08,0.08
runtime,2.78,2.88,2.88,2.73
tmax,100.0,100.0,100.0,100.0
